In [ ]:
from mewpy.omics.expression import ExpressionSet
import numpy as np
import pandas as pd
from src.integration import *
from mewpy import *
from mewpy.omics.integration.gimme import GIMME

At this phase the data is ready to be integrated into a metabolic model. To do so, MewPy will be used, which is a Python library that allows to integrate data into a metabolic model. Two methods were implemented: GIMME and eFlux. To run both of them the gene expression dataset will be necessary TPM file to be integrated in a in house GEM.

Load your model. It will give you a brief description of the model, and also understand if the model was properly loaded.

In [10]:
from src.integration import Integration

my_model = Integration(model="data/inputs/model_ngaditana.xml")

72520 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Model loaded
Objective
1.0 e_Biomass__cytop = 1904.504803688504

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra 7.011E+04         0   0.00%
C00009__extr EX_C00009__dra      2499         0   0.00%
C00011__extr EX_C00011__dra 9.842E+04         1 100.00%
C00059__extr EX_C00059__dra     177.3         0   0.00%
C00080__extr EX_C00080__dra      2676         0   0.00%
C00205__extr EX_C00205__dra     1E+06         0   0.00%
C00244__extr EX_C00244__dra 1.611E+04         0   0.00%
C00305__extr EX_C00305__dra     187.2         0   0.00%
C14818__extr EX_C14818__dra     0.136         0   0.00%

Secretion
---------
      Metabolite           Reaction       Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo -2.655E+05         0   0.00%
 photon680__chlo DM_photon680__chlo -6.486E+04         0   0.00%
    C00007__extr     EX_C0000

In [ ]:
expr = pd.read_csv('data/inputs/wt_tpm.tsv', sep='\t') # load expression data
expr.columns = ["Geneid", 'tpm']
expr["Geneid"] = expr["Geneid"] + "_RA" # add a suffix to the geneid to avoid conflicts with the model
n_genes = expr.shape[0] # number of genes
print("Number of genes:", n_genes)
print("Number of samples:", expr.shape[1]-1)
print("Head of the expression data:")
print(expr.head())
print("\n")
print("Summary of expression data:")
print(expr.describe())

In [ ]:
identifiers = expr['Geneid'].tolist() # list of gene identifiers
conditions = ['tpm'] # list of conditions
gimme_res = my_model.gimme(expr=expr, condition=conditions, local_path="data/outputs/gimme/", file_name= 'gimme')

In [ ]:
gimme_res.to_dataframe().to_csv("data/outputs/gimme/nd.tsv", sep="\t")

In [ ]:
def integrate_multiple_expr_gimme(expr_list):
    for expr_name in expr_list:
        expr = pd.read_csv(f'data/inputs/{expr_name}', sep='\t') # load expression data
        expr.columns = ["Geneid", 'tpm']
        expr["Geneid"] = expr["Geneid"] + "_RA" # add a suffix to the geneid to avoid conflicts with the model
        n_genes = expr.shape[0] # number of genes
        print("Number of genes:", n_genes)
        print("Number of samples:", expr.shape[1]-1)
        print("Head of the expression data:")
        print(expr.head())
        print("\n")
        print("Summary of expression data:")
        print(expr.describe())
        identifiers = expr['Geneid'].tolist() # list of gene identifiers
        conditions = ['tpm'] # list of conditions
        gimme_res = my_model.gimme(expr=expr, conditions=conditions) # local_path="data/outputs/gimme/", file_name= 'gimme')
        out_path = expr_name.replace("_tpm", "")
        gimme_res.to_dataframe().to_csv(f"data/outputs/gimme/{out_path}", sep="\t")
        
def integrate_multiple_expr_eflux(expr_list):
    for expr_name in expr_list:
        expr = pd.read_csv(f'data/inputs/{expr_name}', sep='\t') # load expression data
        expr.columns = ["Geneid", 'tpm']
        expr["Geneid"] = expr["Geneid"] + "_RA" # add a suffix to the geneid to avoid conflicts with the model
        n_genes = expr.shape[0] # number of genes
        print("Number of genes:", n_genes)
        print("Number of samples:", expr.shape[1]-1)
        print("Head of the expression data:")
        print(expr.head())
        print("\n")
        print("Summary of expression data:")
        print(expr.describe())
        identifiers = expr['Geneid'].tolist() # list of gene identifiers
        conditions = ['tpm'] # list of conditions
        eflux_res = my_model.eflux(expr=expr, conditions=conditions) #local_path="data/outputs/eflux/", file_name= 'eflux', parsimonious=True)
        out_path = expr_name.replace("_tpm", "")
        eflux_res.dataframe.to_csv(f"data/outputs/eflux/{out_path}", sep="\t")


In [8]:
eflux_res = integrate_multiple_expr_eflux(["wt_tpm.tsv", "nd_tpm.tsv"])

NameError: name 'integrate_multiple_expr_eflux' is not defined

In [ ]:
eflux_res

In [ ]:
eflux_res

In [3]:
nd = pd.read_csv("data/outputs/gimme/nd.tsv", sep="\t")
wt = pd.read_csv("data/outputs/gimme/wt.tsv", sep="\t")

In [4]:
expr_nd = pd.read_csv(f'data/inputs/nd_tpm.tsv', sep='\t') # load expression data
expr_nd.columns = ["Geneid", 'tpm']
expr_wt = pd.read_csv(f'data/inputs/wt_tpm.tsv', sep='\t') # load expression data
expr_wt.columns = ["Geneid", 'tpm']

In [7]:
expr_nd.head(), expr_wt.head(), nd.head(), wt.head()

(    Geneid       tpm
 0  Ng00001  0.000000
 1  Ng00002  0.000000
 2  Ng00003  0.000000
 3  Ng00004  0.184044
 4  Ng00005  0.243088,
     Geneid       tpm
 0  Ng00001  0.000000
 1  Ng00002  0.000000
 2  Ng00003  0.000000
 3  Ng00004  0.000000
 4  Ng00005  1.595594,
                 Unnamed: 0        value
 0             R02434__chlo     0.000000
 1  T_L_Tyrosine_V2__mitmem     0.087430
 2    TR0011455_PLAS__ermem     0.000000
 3            R01652__cytop     0.000000
 4             R00243__mito -6796.166165,
                 Unnamed: 0        value
 0             R02434__chlo     0.000000
 1  T_L_Tyrosine_V2__mitmem     0.087430
 2    TR0011455_PLAS__ermem     0.000000
 3            R01652__cytop     0.000000
 4             R00243__mito -6796.166165)

In order to run GIMME or eFlux, you need to set your expression data.

In [ ]:
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t') # load expression data

expr["Geneid"] = expr["Geneid"] + "_RA" # add a suffix to the geneid to avoid conflicts with the model
n_genes = expr.shape[0] # number of genes
print("Number of genes:", n_genes)
print("Number of samples:", expr.shape[1]-1)
print("Head of the expression data:")
print(expr.head())
print("\n")
print("Summary of expression data:")
print(expr.describe())

Set the desired solver to be used. In this case the Gurobi solver will be used.

In [ ]:
import mewpy.solvers

mewpy.solvers.set_default_solver('gurobi')
print(mewpy.solvers.get_default_solver())

In [ ]:
my_model.gimme(expr=set_expression, condition=conditions, local_path="data/outputs/gimme/", file_name= 'gimme')

In [ ]:
my_model.eflux(set_expression, local_path="data/outputs/gimme/", file_name= 'eflux')

In [ ]:
#identifiers: list = expr['Geneid'].tolist()
#conditions: list = ['tpm']
#expression = expr['tpm'].to_numpy()[:, np.newaxis]

#set_expression = ExpressionSet(identifiers, conditions, expression)

In [ ]:
integration = Integration("data/inputs/model_ngaditana.xml")

In [ ]:
expr = pd.read_csv('data/inputs/tpm.tsv', sep='\t')
expr["Geneid"] = expr["Geneid"] + "_RA"
condition = ['tpm']

In [ ]:
gimme_res = integration.gimme(expr, condition, "data/results", "gimme_results")

In [ ]:
solution = gimme_res.to_dataframe()

In [ ]:
eflux = integration.eflux(expr, "data/results", "gimme_results", condition)

In [ ]:
eflux

In [ ]:
eflux

In [1]:
import pandas as pd
from integration import Integration

In [2]:
nd_model = pd.read_csv('data/inputs/pfba/nd_model.xml', sep='\t') # load expression data

In [3]:
nd_model = Integration("data/inputs/pfba/nd_model.xml")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Model loaded
Objective
1.0 e_Biomass__cytop = 0.009950042454122277

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         0   0.00%
C00009__extr EX_C00009__dra   0.01306         0   0.00%
C00011__extr EX_C00011__dra    0.5142         1 100.00%
C00059__extr EX_C00059__dra 0.0009262         0   0.00%
C00080__extr EX_C00080__dra   0.01398         0   0.00%
C00205__extr EX_C00205__dra     11.09         0   0.00%
C00244__extr EX_C00244__dra    0.0891         0   0.00%
C00305__extr EX_C00305__dra  0.000978         0   0.00%
C14818__extr EX_C14818__dra 7.107E-07         0   0.00%

Secretion
---------
      Metabolite           Reaction      Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo    -2.945         0   0.00%
 photon680__chlo DM_photon680__chlo   -0.7209         0   0.00%
    C00007__extr     EX_C0000

In [4]:
nd_model.pFBA()

2.42861286636753e-17

In [16]:
my_model.fva()

FVA finds the ranges of each metabolic flux at the optimum.
                               minimum       maximum
R02434__chlo                  0.000000      0.000000
T_L_Tyrosine_V2__mitmem -548343.564066      0.097144
TR0011455_PLAS__ermem         0.000000      0.000000
R01652__cytop                 0.000000  12666.539865
R00243__mito            -561792.117499  13128.334099
PKS_C204__cytop               0.000000    161.567387
EX_C00188__dra                0.000000      0.000000
EX_C05839__dra                0.000000      0.000000
R03362__cytop                 0.000000  66546.188139
R04537__cytop                 0.000000   1823.094117


AttributeError: DictList has no attribute or entry FRD7

In [5]:
wt_model = Integration("data/inputs/pfba/wt_model.xml")
wt_model.pFBA()

Model loaded
Objective
1.0 e_Biomass__cytop = 0.010041327328290849

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra    0.4504         0   0.00%
C00009__extr EX_C00009__dra   0.01318         0   0.00%
C00011__extr EX_C00011__dra    0.5292         1 100.00%
C00059__extr EX_C00059__dra 0.0009347         0   0.00%
C00080__extr EX_C00080__dra   0.03222         0   0.00%
C00205__extr EX_C00205__dra     11.67         0   0.00%
C00244__extr EX_C00244__dra   0.08854         0   0.00%
C00305__extr EX_C00305__dra  0.000987         0   0.00%
C14818__extr EX_C14818__dra 7.173E-07         0   0.00%

Secretion
---------
      Metabolite            Reaction      Flux  C-Number C-Flux
photon298__cytop DM_photon298__cytop  -0.03192         0  0.00%
 photon450__chlo  DM_photon450__chlo   -0.7258         0  0.00%
 photon490__chlo  DM_photon490__chlo   -0.9142         0  0.00%
 photon646__chlo  DM_photon646__chlo     -2.54         0  0.00%
 photon673__chlo 

6.071532165918825e-17

In [7]:
import cobra
from cobra.flux_analysis import flux_variability_analysis
wt_model = cobra.io.read_sbml_model("data/inputs/pfba/wt_model.xml")

Model deleted


In [8]:
wt_model.optimize() # optimize the model

,fluxes,reduced_costs
R02434__chlo,0.000000e+00,0.000000
T_L_Tyrosine_V2__mitmem,5.121830e-07,0.000000
TR0011455_PLAS__ermem,0.000000e+00,0.000000
R01652__cytop,0.000000e+00,0.000000
R00243__mito,-1.522511e-02,-0.052294
...,...,...
R05165__cytop,0.000000e+00,0.000000
R08270__chlo,0.000000e+00,0.000000
T_HMGCOAtm__mitmem,0.000000e+00,-0.000000
EX_C00009__dra,-1.317618e-02,0.000000


In [9]:
flux_variability_analysis(wt_model, ) # perform FVA

TypeError: flux_variability_analysis() got an unexpected keyword argument 'objective_sense'

In [10]:
reaction_list = "data/inputs/fva/nd.tsv" # load reaction list

In [11]:
reaction_list = pd.read_csv(reaction_list, sep='\t') # load reaction list

In [12]:
reaction_list.head()

,Unnamed: 0,fluxes,reduced_costs
0,R02434__chlo,0.000000e+00,-2.000000
1,T_L_Tyrosine_V2__mitmem,5.075268e-07,-2.000000
2,TR0011455_PLAS__ermem,0.000000e+00,-2.000000
3,R01652__cytop,0.000000e+00,-2.000000
4,R00243__mito,-3.953082e-03,854.988265


In [17]:
nd_model = cobra.io.read_sbml_model("data/inputs/pfba/nd_model.xml")
# make reaction list as dataframe
reaction_list = pd.DataFrame(reaction_list)
final_fva = flux_variability_analysis(nd_model, reaction_list=reaction_list) # perform FVA

TypeError: unhashable type: 'DataFrame'

In [3]:
import pandas as pd
nd_tsv = "data/inputs/fva/nd.tsv"
wt_tsv = "data/inputs/fva/wt.tsv"

# as data frame tsv
nd_tsv = pd.read_csv(nd_tsv, sep='\t')

In [4]:
from cobra.io import load_model
model = load_model('textbook')nd_tsv.head()

,Unnamed: 0,fluxes,reduced_costs
0,R02434__chlo,0.000000e+00,-2.000000
1,T_L_Tyrosine_V2__mitmem,5.075268e-07,-2.000000
2,TR0011455_PLAS__ermem,0.000000e+00,-2.000000
3,R01652__cytop,0.000000e+00,-2.000000
4,R00243__mito,-3.953082e-03,854.988265


In [3]:
import cobra.io
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
nd_model = cobra.io.read_sbml_model("data/inputs/fva/nd_model.xml")
nd_constraints = pd.read_csv("data/inputs/pfba/nd.tsv", sep='\t')



In [6]:
# add constraints to the model






TypeError: unhashable type: 'DataFrame'

In [9]:
# add constraints to the model
nd_model.add_cons_vars(nd_constraints)

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
nd_model.optimize() # optimize the model

In [ ]:

flux_variability_analysis(nd_model) # perform FVA

In [5]:
import cobra
import pandas as pd
from src.integration import Integration

wt_model = Integration("data/inputs/fva/wt_model.xml")
wt_model.pFBA()



Model loaded
Objective
1.0 e_Biomass__cytop = 0.010041327328290849

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra    0.4504         0   0.00%
C00009__extr EX_C00009__dra   0.01318         0   0.00%
C00011__extr EX_C00011__dra    0.5292         1 100.00%
C00059__extr EX_C00059__dra 0.0009347         0   0.00%
C00080__extr EX_C00080__dra   0.03222         0   0.00%
C00205__extr EX_C00205__dra     11.67         0   0.00%
C00244__extr EX_C00244__dra   0.08854         0   0.00%
C00305__extr EX_C00305__dra  0.000987         0   0.00%
C14818__extr EX_C14818__dra 7.173E-07         0   0.00%

Secretion
---------
      Metabolite            Reaction      Flux  C-Number C-Flux
photon298__cytop DM_photon298__cytop  -0.03192         0  0.00%
 photon450__chlo  DM_photon450__chlo   -0.7258         0  0.00%
 photon490__chlo  DM_photon490__chlo   -0.9142         0  0.00%
 photon646__chlo  DM_photon646__chlo     -2.54         0  0.00%
 photon673__chlo 

6.071532165918825e-17

In [6]:
import cobra
import pandas as pd
from src.integration import Integration

nd_model = Integration("data/inputs/fva/nd_model.xml")
nd_model.pFBA()





Model loaded
Objective
1.0 e_Biomass__cytop = 0.009950042454122277

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         0   0.00%
C00009__extr EX_C00009__dra   0.01306         0   0.00%
C00011__extr EX_C00011__dra    0.5142         1 100.00%
C00059__extr EX_C00059__dra 0.0009262         0   0.00%
C00080__extr EX_C00080__dra   0.01398         0   0.00%
C00205__extr EX_C00205__dra     11.09         0   0.00%
C00244__extr EX_C00244__dra    0.0891         0   0.00%
C00305__extr EX_C00305__dra  0.000978         0   0.00%
C14818__extr EX_C14818__dra 7.107E-07         0   0.00%

Secretion
---------
      Metabolite           Reaction      Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo    -2.945         0   0.00%
 photon680__chlo DM_photon680__chlo   -0.7209         0   0.00%
    C00007__extr     EX_C00007__dra    -0.184         0   0.00%
    C00014__extr     EX_C00014__dra  -0.00492         0   0.00%
    C00027__extr 

2.42861286636753e-17

In [ ]:
import cobra
import pandas as pd
from src.integration import Integration

wt_model = Integration("data/inputs/fva/wt_model.xml")
wt_model.fva()

In [ ]:
import cobra
import pandas as pd
from src.integration import Integration

wt_model = Integration("data/inputs/fva/wt_model.xml")
wt_model.fva()

In [2]:
import cobra
import pandas as pd


nd_model = cobra.io.read_sbml_model("data/outputs/gems/nd_model.xml")
#nd_model.fva()

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [10]:
nd_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,1.409,0,0.00%
C00009__extr,EX_C00009__dra,0.01306,0,0.00%
C00011__extr,EX_C00011__dra,0.5142,1,100.00%
C00059__extr,EX_C00059__dra,0.0009262,0,0.00%
C00080__extr,EX_C00080__dra,0.01398,0,0.00%
C00205__extr,EX_C00205__dra,11.09,0,0.00%
C00244__extr,EX_C00244__dra,0.0891,0,0.00%
C00305__extr,EX_C00305__dra,0.000978,0,0.00%
C14818__extr,EX_C14818__dra,7.107E-07,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [ ]:
metabolite_id_dic = {}
for metabolite in nd_model.metabolites:
    metabolite_id_dic[metabolite.id] = metabolite.name, metabolite.formula

for id in metabolite_id_dic:
    print(id, metabolite_id_dic[id])

In [ ]:

lipids = ['Fatty acid', ]
pigments = []
others_of_interest = ['L-Carnitine', 'L-Glutamine', 'L-Glutamic acid', 'L-Histidine', 'L-Lactate', 'L-Leucine', 'L-Lysine', 'L-Methionine', 'L-Phenylalanine', 'L-Proline', 'L-Serine', 'L-Threonine', 'L-Tryptophan', 'L-Tyrosine', 'L-Valine', 'L-Arginine', 'L-Asparagine', 'L-Aspartate', 'L-Cysteine', 'L-Glutamate', 'L-Glycine', 'L-Histidine', 'L-Isoleucine', 'L-Leucine', 'L-Lysine', 'L-Methionine', 'L-Phenylalanine', 'L-Proline', 'L-Serine', 'L-Threonine', 'L-Tryptophan', 'L-Tyrosine', 'L-Valine', 'L-Arginine', 'L-Asparagine', 'L-Aspartate', 'L-Cysteine', 'L-Glutamate', 'L-Glycine', 'L-Histidine', 'L-Isoleucine', 'L-Leucine', 'L-Lysine', 'L-Methionine', 'L-Phenylalanine', 'L-Proline', 'L-Serine', 'L-Threonine', 'L-Tryptophan', 'L-Tyrosine', 'L-Valine', 'L-Arginine', 'L-Asparagine', 'L-Aspartate', 'L-Cysteine', 'L-Glutamate', 'L-Glycine', 'L-Histidine', 'L-Isoleucine', 'L-Leucine', 'L-Lysine', 'L-Methionine', 'L-Phenylalanine', 'L-Proline', 'L-Serine', 'L-Threonine', 'L-Tryptophan', 'L-Tyrosine', 'L-Valine', 'L-Arginine', 'L-Asparagine', 'Morphine', 'Vitamine D-3' , 'Vitamine D-5']

In [54]:
import re
for metabolite in nd_model.metabolites:
    if re.search("omega", metabolite.name) or re.search("Omega", metabolite.name):
        print(metabolite.name, metabolite.id, metabolite.formula)


omega-(Methylthio)alkyl-glucosinolate C21691__chlo C11H21NO9S3
omega-(Methylsulfinyl)alkyl-glucosinolate C21692__chlo C11H21NO10S3


In [ ]:
for metabolite in nd_model.metabolites:
    if re.search("acid", metabolite.name):
        print(metabolite.name, metabolite.id, metabolite.formula)


In [46]:
# if lipid or Lipid in metabolite.name, then it is a lipid
pufas = ['Octadecatrienoic acid', ]

In [ ]:
#Lipids with 14-20 carbons are usually used for biofuels production
# example of how formula is displayed: C24H39O5
# number of carbons in chain: 14-20
for metabolite in nd_model.metabolites:
    # After C it has to be a number between 14 and 20:
    if re.search("^C\d{1,2}", metabolite.formula):
        print(metabolite.name, metabolite.id, metabolite.formula)

In [ ]:
#Lipids with 14-20 carbons are usually used for biofuels production
# example of how formula is displayed: C24H39O5
# number of carbons in chain: 14-20
for metabolite in nd_model.metabolites:
    # After C it has to be a number between 14 and 20:
    if re.search("^C\d[1-2]", metabolite.formula):
        print(metabolite.name, metabolite.id, metabolite.formula)


In [ ]:
print('\n')
print('Running FVA in summary methods')
        print('Model summary. default fva=0.95')
        print(self.model.summary(fva=fva))
        print('Variability in metabolite mass balances with flux variability analysis:')
        #search for omega-3 fatty acids
        print(self.model.metabolites.get_by_id('omega3_fatty_acids').summary(fva=fva))
        print(self.model.metabolites.omega.summary(fva=fva))
        print('Variability in reaction fluxes with flux variability analysis:')
        print(self.model.reactions.EX_pyr_c.summary(fva=fva))

In [89]:
#Lipids with 14-20 carbons are usually used for biofuels production
# example of how formula is displayed: C24H39O5
# number of carbons in chain: 14-20

for metabolite in nd_model.metabolites:
    # After C it has to be a number between 14 and 20:
    shadow_price = metabolite.shadow_price
    # grab the metabolites that have formula beggining formula like C14, C15, C16, C17, C18, C19, C20
    if re.search("


SyntaxError: EOL while scanning string literal (1400732583.py, line 9)

In [98]:
print(nd_model.constraints)

In [101]:
pufa = ['Linoleic acid', 'alpha-Linolenic acid', 'gamma-Linolenic acid', 'Columbinic acid', 'Stearidonic acid',
                'Mead acid', 'Dihomo-γ-linolenic acid', 'Arachidonic acid', 'Eicosapentaenoic acid', 'Docosapentaenoic '
                                                                                                     'acid',
                'Docosahexaenoic acid']

In [ ]:
import re

for metabolite in nd_model.metabolites:
    if re.search(('C[^1]\d'), metabolite.formula):
        print(metabolite.name, metabolite.id, metabolite.formula)

In [113]:
name = 'Fatty'

for metabolite in nd_model.metabolites:
    if name in metabolite.name:
        print(metabolite.name, metabolite.id, metabolite.formula)

Fatty acid C00162__extr CO2R
Fatty acid C00162__mito CO2R
Fatty acid C00162__e_r_ CO2R
Fatty acid C00162__cytop CO2R
Fatty acid C00162__chlo CO2R


In [116]:
id = 'C00162__chlo'

for metabolite in nd_model.metabolites:
    if id in metabolite.id:
        print(metabolite.name, metabolite.id, metabolite.formula, metabolite.shadow_price)

Fatty acid C00162__chlo CO2R 12730.712342111896


In [ ]:
for metabolite in nd_model.metabolites:
    print(metabolite.reactions)

In [128]:
metabolite_in_study = 'beta-Tocopherol'

nd_model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,1.409,0,0.00%
C00009__extr,EX_C00009__dra,0.01306,0,0.00%
C00011__extr,EX_C00011__dra,0.5142,1,100.00%
C00059__extr,EX_C00059__dra,0.0009262,0,0.00%
C00080__extr,EX_C00080__dra,0.01398,0,0.00%
C00205__extr,EX_C00205__dra,11.09,0,0.00%
C00244__extr,EX_C00244__dra,0.0891,0,0.00%
C00305__extr,EX_C00305__dra,0.000978,0,0.00%
C14818__extr,EX_C14818__dra,7.107E-07,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [ ]:
list_pigments = ['zeaxanthin', 'Zeaxanthin', 'canthaxanthin', 'Canthaxanthin', 'Astaxanthin', 'astaxanthin',
                 'Chlorophyll a', 'chlorophyll a', 'Chlorophyll b', 'chlorophyll b', 'Chlorophyll c', 'chlorophyll c', 'beta-Carotene', 'Violaxanthin', 'violaxanthin', 'Vaucheriaxanthin', 'vaucheriaxanthin''canthaxanthin, Canthaxanthin', 'carotenoid', 'Carotenoid']




for metabolite in nd_model.metabolites:
    if pigment or or_pigment in metabolite.name:
        print(metabolite.name, metabolite.id, metabolite.formula)

In [130]:
search_for_reaction(nd_model, '')

In [134]:
metabolite_id = 'C00237__extr'
search_for_reaction(nd_model, metabolite_id)



In [6]:
from src.integration import Integration

nd_model = Integration('data/inputs/fva/nd_model.xml')

Model loaded
Objective
1.0 e_Biomass__cytop = 0.009950042454122277

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         0   0.00%
C00009__extr EX_C00009__dra   0.01306         0   0.00%
C00011__extr EX_C00011__dra    0.5142         1 100.00%
C00059__extr EX_C00059__dra 0.0009262         0   0.00%
C00080__extr EX_C00080__dra   0.01398         0   0.00%
C00205__extr EX_C00205__dra     11.09         0   0.00%
C00244__extr EX_C00244__dra    0.0891         0   0.00%
C00305__extr EX_C00305__dra  0.000978         0   0.00%
C14818__extr EX_C14818__dra 7.107E-07         0   0.00%

Secretion
---------
      Metabolite           Reaction      Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo    -2.945         0   0.00%
 photon680__chlo DM_photon680__chlo   -0.7209         0   0.00%
    C00007__extr     EX_C00007__dra    -0.184         0   0.00%
    C00014__extr     EX_C00014__dra  -0.00492         0   0.00%
    C00027__extr 

In [2]:
nd_model.pigments()

Potential pigment: Red chlorophyll catabolite, id: C18022__mito, formula: C35H36N4O7,  shadow price: 0.0
Potential pigment: Red chlorophyll catabolite, id: C18022__cytop, formula: C35H36N4O7,  shadow price: 1.6063184916900495
Potential pigment: Chlorophyll b, id: C05307__chlo, formula: C55H70MgN4O6,  shadow price: 0.0
Potential pigment: beta-Carotene, id: C02094__chlo, formula: C40H56,  shadow price: 3973.0604668354745
Potential pigment: 9-cis-beta-Carotene, id: C20484__chlo, formula: C40H56,  shadow price: 3972.0604668354745
Potential pigment: Chlorophyll a, id: C05306__chlo, formula: C55H73MgN4O5,  shadow price: 6924.930348846686
Potential pigment: Violaxanthin, id: C08614__chlo, formula: C40H56O4,  shadow price: 3830.50882303492
Potential pigment: Red chlorophyll catabolite, id: C18022__chlo, formula: C35H36N4O7,  shadow price: -32.637910950138576
Potential pigment: Zeaxanthin, id: C06098__chlo, formula: C40H56O2,  shadow price: 3901.7846449351973


In [ ]:
nd_model.pufas()

In [ ]:
nd_model.biodiesel_fatty_acids()

In [ ]:
import cobra

# load model
nd_model = cobra.io.read_sbml_model('data/inputs/fva/nd_model.xml')

# see reactions in model
for reaction in nd_model.reactions:
    print(reaction.id, reaction.name)

In [13]:
words = ['fatty', 'Fatty', 'Lipid', 'lipid']

for reaction in nd_model.reactions:
    for word in words:
        if word in reaction.name:
            print(reaction.id, reaction.name)

R02250__cytop Triacylglycerol + H2O => 1,2-Diacyl-sn-glycerol + H+ + Fatty acid
R00390__mito ATP + CoA + Fatty acid => Diphosphate + AMP + Acyl-CoA + H+
e_Lipid__cytop 0.1 Phosphatidylglycerol (chlo) + 0 Phosphatidylethanolamine (cytop) + 0 1,2-Diacyl-sn-glycerol (e.r.) + 0.4 Triacylglycerol (e.r.) + 0.1 Diacylglyceryl-N,N,N-trimethylhomoserine (e.r.) + 0.1 Digalactosyl-diacylglycerol (chlo) + 0.1 Sulfoquinovosyldiacylglycerol (chlo) + 0.1 Phosphatidylcholine (cytop) + 0 Fatty acid (cytop) + 0 1-Phosphatidyl-D-myo-inositol (cytop) + 0.2 1,2-Diacyl-3-beta-D-galactosyl-sn-glycerol (chlo) => e-Lipid (cytop)
e_Lipid__cytop 0.1 Phosphatidylglycerol (chlo) + 0 Phosphatidylethanolamine (cytop) + 0 1,2-Diacyl-sn-glycerol (e.r.) + 0.4 Triacylglycerol (e.r.) + 0.1 Diacylglyceryl-N,N,N-trimethylhomoserine (e.r.) + 0.1 Digalactosyl-diacylglycerol (chlo) + 0.1 Sulfoquinovosyldiacylglycerol (chlo) + 0.1 Phosphatidylcholine (cytop) + 0 Fatty acid (cytop) + 0 1-Phosphatidyl-D-myo-inositol (cytop) + 0.

In [16]:
nd_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,1.409,0,0.00%
C00009__extr,EX_C00009__dra,0.01306,0,0.00%
C00011__extr,EX_C00011__dra,0.5142,1,100.00%
C00059__extr,EX_C00059__dra,0.0009262,0,0.00%
C00080__extr,EX_C00080__dra,0.01398,0,0.00%
C00205__extr,EX_C00205__dra,11.09,0,0.00%
C00244__extr,EX_C00244__dra,0.0891,0,0.00%
C00305__extr,EX_C00305__dra,0.000978,0,0.00%
C14818__extr,EX_C14818__dra,7.107E-07,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [ ]:
for metabolite in nd_model.metabolites:
    for reaction in metabolite.reactions:
        print(metabolite.name, reaction.name, reaction.id, reaction.lower_bound, reaction.upper_bound)

In [9]:
for metabolite in nd_model.metabolites:
    if 'Fatty' in metabolite.name:
        print(metabolite.name, metabolite.id, metabolite.formula)

Fatty acid C00162__extr CO2R
Fatty acid C00162__mito CO2R
Fatty acid C00162__e_r_ CO2R
Fatty acid C00162__cytop CO2R
Fatty acid C00162__chlo CO2R
Model loaded
Objective
1.0 e_Biomass__cytop = 0.009950042454122277

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         0   0.00%
C00009__extr EX_C00009__dra   0.01306         0   0.00%
C00011__extr EX_C00011__dra    0.5142         1 100.00%
C00059__extr EX_C00059__dra 0.0009262         0   0.00%
C00080__extr EX_C00080__dra   0.01398         0   0.00%
C00205__extr EX_C00205__dra     11.09         0   0.00%
C00244__extr EX_C00244__dra    0.0891         0   0.00%
C00305__extr EX_C00305__dra  0.000978         0   0.00%
C14818__extr EX_C14818__dra 7.107E-07         0   0.00%

Secretion
---------
      Metabolite           Reaction      Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo    -2.945         0   0.00%
 photon680__chlo DM_photon680__chlo   -0.7209         0   0.00%

In [ ]:
from src.integration import Integration
integration = Integration('data/outputs/gems/nd_model.xml')

In [10]:
integration.pFBA()

2.42861286636753e-17

In [11]:
integration.fva()

FVA finds the ranges of each metabolic flux at the optimum.
                              minimum       maximum
R02434__chlo             0.000000e+00  0.000000e+00
T_L_Tyrosine_V2__mitmem -8.561880e-04  4.697441e-07
TR0011455_PLAS__ermem    0.000000e+00  0.000000e+00
R01652__cytop           -3.840839e-19  2.226145e-03
R00243__mito            -3.953082e-03  3.953082e-03
PKS_C204__cytop         -3.527767e-19  8.440867e-04
EX_C00188__dra           0.000000e+00  0.000000e+00
EX_C05839__dra           0.000000e+00  0.000000e+00
R03362__cytop            0.000000e+00  6.682018e-03
R04537__cytop            0.000000e+00  6.281092e-03


Running FVA in summary methods
Model summary. default fva=0.95
Objective
1.0 e_Biomass__cytop = 0.009950042454079412

Uptake
------
  Metabolite       Reaction      Flux                  Range  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         [1.122; 1.462]         0   0.00%
C00009__extr EX_C00009__dra   0.01306      [0.0124; 0.01423]         0   0.0

KeyError: 'omega3_fatty_acids'

In [1]:
from integration import Integration
integration_wt = Integration('data/outputs/gems/wt_model.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Model loaded
Objective
1.0 e_Biomass__cytop = 0.010041327328290849

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra    0.4504         0   0.00%
C00009__extr EX_C00009__dra   0.01318         0   0.00%
C00011__extr EX_C00011__dra    0.5292         1 100.00%
C00059__extr EX_C00059__dra 0.0009347         0   0.00%
C00080__extr EX_C00080__dra   0.03222         0   0.00%
C00205__extr EX_C00205__dra     11.67         0   0.00%
C00244__extr EX_C00244__dra   0.08854         0   0.00%
C00305__extr EX_C00305__dra  0.000987         0   0.00%
C14818__extr EX_C14818__dra 7.173E-07         0   0.00%

Secretion
---------
      Metabolite            Reaction      Flux  C-Number C-Flux
photon298__cytop DM_photon298__cytop  -0.03192         0  0.00%
 photon450__chlo  DM_photon450__chlo   -0.7258         0  0.00%
 photon490__chlo  DM_photon49

In [4]:
integration_wt.pFBA('wt_pfba','data/outputs/pfba/')

In [2]:
integration_wt.fva('wt_fva','data/outputs/fva/')

FVA finds the ranges of each metabolic flux at the optimum.


KeyError: 'R02434__chlo'

In [3]:
integration_nd = Integration('data/outputs/gems/nd_model.xml')

Model loaded
Objective
1.0 e_Biomass__cytop = 0.009950042454122277

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     1.409         0   0.00%
C00009__extr EX_C00009__dra   0.01306         0   0.00%
C00011__extr EX_C00011__dra    0.5142         1 100.00%
C00059__extr EX_C00059__dra 0.0009262         0   0.00%
C00080__extr EX_C00080__dra   0.01398         0   0.00%
C00205__extr EX_C00205__dra     11.09         0   0.00%
C00244__extr EX_C00244__dra    0.0891         0   0.00%
C00305__extr EX_C00305__dra  0.000978         0   0.00%
C14818__extr EX_C14818__dra 7.107E-07         0   0.00%

Secretion
---------
      Metabolite           Reaction      Flux  C-Number  C-Flux
 photon646__chlo DM_photon646__chlo    -2.945         0   0.00%
 photon680__chlo DM_photon680__chlo   -0.7209         0   0.00%
    C00007__extr     EX_C00007__dra    -0.184         0   0.00%
    C00014__extr     EX_C00014__dra  -0.00492         0   0.00%
    C00027__extr 

In [4]:
integration_nd.pFBA('nd_pfba','data/outputs/pfba/')

pFBA solution saved as csv file at data/outputs/pfba/


In [5]:
integration_nd.fva('nd_fva','data/outputs/fva/')

FVA finds the ranges of each metabolic flux at the optimum.


KeyError: 'R02434__chlo'

In [6]:
import cobra

nd_model = cobra.io.read_sbml_model('data/outputs/gems/nd_model.xml')

wt_model = cobra.io.read_sbml_model('data/outputs/gems/wt_model.xml')

In [12]:
fva_nd_model = cobra.flux_analysis.flux_variability_analysis(nd_model)
fva_wt_model = cobra.flux_analysis.flux_variability_analysis(wt_model)

pfba_nd_model = cobra.flux_analysis.parsimonious.pfba(nd_model)
pfba_wt_model = cobra.flux_analysis.parsimonious.pfba(wt_model)

In [8]:
# lower bound reaction EX_C00205__dra is -200

models = [wt_model, nd_model]
for model in models:
    for reaction in model.reactions:
        if reaction.id.startswith("PRISM") and 'PRISM_solar_exo__extr' not in reaction.id:
            reaction.bounds = (0,0)
        if reaction.id == "PRISM_solar_exo__extr":
            reaction.upper_bound = 1000
        if reaction.id.startswith("DM_"):
            reaction.upper_bound = 1000

In [9]:
import pandas as pd
import csv as csv

file_name = 'nd_fva_1.csv'
path = 'data/outputs/fva/'

#save fva solutions as csv
with open(path + file_name, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Reaction', 'Lower bound', 'Upper bound'])
    for reaction in nd_model.reactions:
        writer.writerow([reaction.id, reaction.lower_bound, reaction.upper_bound])

In [10]:
file_name = 'wt_fva_1.csv'
path = 'data/outputs/fva/'

with open(path + file_name, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Reaction', 'Lower bound', 'Upper bound'])
    for reaction in wt_model.reactions:
        writer.writerow([reaction.id, reaction.lower_bound, reaction.upper_bound])

In [51]:
data = pd.DataFrame()
data['flux_wt'] = pfba_wt_model.fluxes / pfba_wt_model.fluxes['e_Biomass__cytop']
data['flux_nd'] = pfba_nd_model.fluxes / pfba_wt_model.fluxes['e_Biomass__cytop']

data['ratio'] = data['flux_nd'] / data['flux_wt']


In [53]:
data.head()

,flux_wt,flux_nd,ratio
R02434__chlo,0.000000,0.000000,NaN
T_L_Tyrosine_V2__mitmem,0.000051,0.000051,0.990909
TR0011455_PLAS__ermem,0.000000,0.000000,NaN
R01652__cytop,0.000000,0.000000,NaN
R00243__mito,-1.516245,-0.393681,0.259642


In [54]:
data.to_csv('data/outputs/ratio/pfba_1_study_corrigido.csv')

In [55]:
data.sort_values(by='ratio', ascending=False)

,flux_wt,flux_nd,ratio
R01373__chlo,0.0,8.526641e-02,inf
T_Oxygen_V4__permem,0.0,3.613064e+00,inf
R06286__chlo,0.0,9.739913e-02,inf
T_NADtm__mitmem,0.0,1.133727e-16,inf
PKS_C102__cytop,0.0,2.684760e-01,inf
...,...,...,...
R00893__cytop,0.0,0.000000e+00,NaN
R05165__cytop,0.0,0.000000e+00,NaN
R08270__chlo,0.0,0.000000e+00,NaN
T_HMGCOAtm__mitmem,0.0,0.000000e+00,NaN


In [56]:
data = data.dropna()

In [ ]:
data.sort_values(by='ratio', ascending=False)

In [67]:
data['ratio_100'] = data['ratio'] * 100

In [ ]:
data.sort_values(by='ratio_100', ascending=False)

In [96]:
data['ratio'] = abs(data['ratio'])

In [ ]:
data = data.loc[round(data["flux_wt"],4) != round(data["flux_nd"],4)]

data

In [99]:
# filter dataframe grab those that have more than 125% at ratio_100 (overexpressed)
data_125 = data[data['ratio_100'] > 125]

# grab those that have less than 75% at ratio_100 (underexpressed)
data_75 = data[data['ratio_100'] < 75]

In [74]:
# sort matrix by ratio_100
data_125 = data_125.sort_values(by='ratio_100', ascending=False)

In [100]:
data_75 = data_75.sort_values(by='ratio_100', ascending=False)

In [101]:
# save both files

data_125.to_csv('data/outputs/ratio/pfba_1_study_overexpressed.csv')
data_75.to_csv('data/outputs/ratio/pfba_1_study_underexpressed.csv')

In [ ]:
data_filt = data.loc[(data["ratio"] <25) | (data_diff["ratio"] > 125)]

In [61]:
# save data as csv
data.to_csv('data/outputs/ratio/pfba_1_study_corrigido_sorted.csv')

In [62]:
# grab only those reactions with a ratio greater than 25 at ratio_100
greater_25_new = pd.DataFrame(data[data['ratio_100'] > 25])

In [64]:
# sort the matrix by ratio_100
final_data = greater_25_new.sort_values(by='ratio_100', ascending=False)

In [66]:
# abs ratio
final_data['ratio'] = final_data['ratio'].abs()


In [65]:
# save great final_data
final_data.to_csv('data/outputs/ratio/pfba_study1_final.csv')

In [26]:
#grab index of R00833__mito
data.index.get_loc('R00833__mito')

149

In [32]:
great_25_final = data.iloc[0:150]

In [34]:
data.head()

,flux_wt,flux_nd,ratio,ratio_100
T_L_Tyrosine_V2__mitmem,5.100750e-05,5.054380e-05,1.009174,100.917432
R00243__mito,-1.516245e+00,-3.936812e-01,3.851454,385.145420
PKS_C204__cytop,1.628888e-02,6.284723e-02,0.259182,25.918212
R00848__mito,6.123937e-01,4.711805e-01,1.299701,129.970082
T_CoA___chlomem,4.318960e-16,-1.727584e-16,-2.500000,-250.000000


In [37]:
data_p = pd.read_csv('data/outputs/ratio/pfba_1_study.csv')
data_p.head()

,Unnamed: 0,flux_wt,flux_nd,ratio
0,R02434__chlo,0.000000,0.000000,NaN
1,T_L_Tyrosine_V2__mitmem,0.000051,0.000051,1.009174
2,TR0011455_PLAS__ermem,0.000000,0.000000,NaN
3,R01652__cytop,0.000000,0.000000,NaN
4,R00243__mito,-1.516245,-0.393681,3.851454


In [48]:
data_new = pd.read_csv('data/outputs/ratio/pfba_1_study.csv')

In [50]:
data_new = data_new.dropna()

In [2]:
def getPatwhaysMap(model):
    res = {}
    for pathway in model.groups:
        for reaction in pathway.members:
            if reaction.id not in res.keys():
                res[reaction.id] = [pathway.name]
            else:
                res[reaction.id].append(pathway.name)
    return res

In [102]:
dic_reactions = getPatwhaysMap(wt_model)

data_overexpressed = pd.read_csv('data/outputs/ratio/pfba_1_study_overexpressed.csv')

data_underexpressed = pd.read_csv('data/outputs/ratio/pfba_1_study_underexpressed.csv')

In [103]:
# list id of reactions that are overexpressed it is the first column of the csv file

overexpressed = []
for i in range(len(data_overexpressed)):
    overexpressed.append(data_overexpressed.iloc[i,0])


In [104]:
len(overexpressed)

195

In [105]:
underexpressed = []
for i in range(len(data_underexpressed)):
    underexpressed.append(data_underexpressed.iloc[i,0])

In [106]:
len(underexpressed)

213

In [107]:
path_map = getPatwhaysMap(wt_model)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in overexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

In [108]:

sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)


[('Transporters pathway', 36), ('Biosynthesis of amino acids', 25), ('Fatty acid metabolism', 20), ('Fatty acid biosynthesis', 20), ('Biosynthesis of unsaturated fatty acids', 19), ('Carbon fixation in photosynthetic organisms', 18), ('PKS system', 18), ('Glycolysis / Gluconeogenesis', 15), ('Pyruvate metabolism', 13), ('Citrate cycle (TCA cycle)', 12), ('Drains pathway', 12), ('Glyoxylate and dicarboxylate metabolism', 8), ('2-Oxocarboxylic acid metabolism', 8), ('Biosynthesis of cofactors', 7), ('Pentose phosphate pathway', 7), ('Pyrimidine metabolism', 7), ('Alanine, aspartate and glutamate metabolism', 6), ('Carbon fixation pathways in prokaryotes', 6), ('Fructose and mannose metabolism', 5), ('Photosynthesis', 5), ('Methane metabolism', 4), ('One carbon pool by folate', 3), ('Glycerophospholipid metabolism', 3), ('Propanoate metabolism', 3), ('Arginine biosynthesis', 3), ('Nitrogen metabolism', 3), ('Cysteine and methionine metabolism', 3), ('Glutathione metabolism', 3), ('Lysine 

In [110]:
path_map = getPatwhaysMap(wt_model)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in underexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)

[('Transporters pathway', 59), ('Fatty acid metabolism', 49), ('Fatty acid biosynthesis', 38), ('Glycolysis / Gluconeogenesis', 16), ('Biosynthesis of amino acids', 16), ('Valine, leucine and isoleucine degradation', 13), ('Pentose phosphate pathway', 12), ('Carbon fixation in photosynthetic organisms', 11), ('Pyruvate metabolism', 10), ('Biosynthesis of unsaturated fatty acids', 10), ('Carbon fixation pathways in prokaryotes', 7), ('Biosynthesis of cofactors', 7), ('2-Oxocarboxylic acid metabolism', 6), ('Citrate cycle (TCA cycle)', 5), ('Alanine, aspartate and glutamate metabolism', 5), ('Phenylalanine, tyrosine and tryptophan biosynthesis', 5), ('Purine metabolism', 5), ('Glyoxylate and dicarboxylate metabolism', 4), ('Amino sugar and nucleotide sugar metabolism', 4), ('Glycine, serine and threonine metabolism', 4), ('Glutathione metabolism', 4), ('Propanoate metabolism', 3), ('Methane metabolism', 3), ('Glycerolipid metabolism', 3), ('Glycerophospholipid metabolism', 3), ('Fructose

In [115]:
# print reactions related to fatty acid biosynthesis at overexpressed list
for i in range(len(overexpressed)):
    if overexpressed[i] in path_map.keys():
        if 'Fatty acid biosynthesis' in path_map[overexpressed[i]]:
            print(overexpressed[i])



R01626__cytop
R00742__cytop
R04958__chlo
R04954__cytop
R04724__chlo
R04966__chlo
R04429__cytop
R04952__cytop
R00390__chlo
R04955__chlo
R04969__chlo
R04533__cytop
R04429__chlo
R04953__cytop
R04428__cytop
R01624__cytop
R04961__chlo
R04355__cytop
R01280__mito
R04955__cytop


In [118]:
under_fatty = []
for i in range(len(underexpressed)):
    if underexpressed[i] in path_map.keys():

        if 'Fatty acid biosynthesis' in path_map[underexpressed[i]]:
            under_fatty.append(underexpressed[i])
            print(underexpressed[i])

R10707__chlo
R04726__chlo
R04957__chlo
R04963__chlo
R04954__chlo
R04952__chlo
R04964__chlo
R04537__chlo
R04953__chlo
R04536__chlo
R04534__chlo
R04568__chlo
R04533__chlo
R04566__chlo
R04960__chlo
R04428__chlo
R04535__chlo
R04965__chlo
R04543__chlo
R04968__chlo
R04544__chlo
R01626__chlo
R00742__chlo
R07763__chlo
R07764__chlo
R07765__chlo
R07762__chlo
R04970__chlo
R04959__chlo
R04725__chlo
R04967__chlo
R04430__chlo
R04956__chlo
R04962__chlo
R08163__chlo
R00390__e_r_
RXN_16380__e_r_
RXN_16380__mito


In [120]:
# new study! ~~~~~
#c5a- control, n5a - nitrogen deprivation, p5a - phosphorus deprivation

In [5]:
import cobra
import pandas as pd
model_c5a = cobra.io.read_sbml_model('data/outputs/gems/c5a_model.xml')
model_n5a = cobra.io.read_sbml_model('data/outputs/gems/n5a_model.xml')
model_p5a = cobra.io.read_sbml_model('data/outputs/gems/p5a_model.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [124]:
fva_c5a = cobra.flux_analysis.flux_variability_analysis(model_c5a)
fva_n5a = cobra.flux_analysis.flux_variability_analysis(model_n5a)
fva_p5a = cobra.flux_analysis.flux_variability_analysis(model_p5a)

pfba_c5a = cobra.flux_analysis.parsimonious.pfba(model_c5a)
pfba_n5a = cobra.flux_analysis.parsimonious.pfba(model_n5a)
pfba_p5a = cobra.flux_analysis.parsimonious.pfba(model_p5a)

In [6]:
models = [model_c5a, model_n5a, model_p5a]
for model in models:
    for reaction in model.reactions:
        if reaction.id.startswith("PRISM") and 'PRISM_solar_exo__extr' not in reaction.id:
            reaction.bounds = (0,0)
        if reaction.id == "PRISM_solar_exo__extr":
            reaction.upper_bound = 1000
        if reaction.id.startswith("DM_"):
            reaction.upper_bound = 1000

In [130]:
fva_c5a.to_csv('data/outputs/fva/fva_c5a.csv')

fva_n5a.to_csv('data/outputs/fva/fva_n5a.csv')

fva_p5a.to_csv('data/outputs/fva/fva_p5a.csv')

In [135]:
# save pfba solution as csv
import pandas as pd
import csv as csv

data2 = pd.DataFrame()
data2['flux_c5a'] = pfba_c5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data2['flux_n5a'] = pfba_n5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data2['flux_p5a'] = pfba_n5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']

In [136]:
data2

,flux_c5a,flux_n5a,flux_p5a
R02434__chlo,0.000000,0.000000,0.000000
T_L_Tyrosine_V2__mitmem,0.000000,0.000051,0.000051
TR0011455_PLAS__ermem,0.000000,0.000000,0.000000
R01652__cytop,0.000000,0.000000,0.000000
R00243__mito,-2.618671,-2.618671,-2.618671
...,...,...,...
R05165__cytop,0.000000,0.000000,0.000000
R08270__chlo,0.000000,0.000000,0.000000
T_HMGCOAtm__mitmem,0.000000,0.000000,0.000000
EX_C00009__dra,-1.312195,-1.311704,-1.311704


In [143]:
data3 = pd.DataFrame()

# flux_c5a vs flux_n5a
data3['flux_c5a'] = pfba_c5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data3['flux_n5a'] = pfba_n5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data3['ratio'] = data3['flux_n5a'] / data3['flux_c5a']

data3['ratio'] = abs(data3['ratio'])
data3['ratio_100'] = data3['ratio'] * 100

In [179]:
# remove NaN values
#data3 = data3.dropna()
data3.head()

,flux_c5a,flux_n5a,ratio,ratio_100
T_L_Tyrosine_V2__mitmem,0.000000,0.000051,inf,inf
R00848__mito,1.372964,1.638747,1.193583,119.358339
T_UDPG__chlomem,-0.754264,-0.881329,1.168463,116.846293
R01626__chlo,7.377894,7.375132,0.999626,99.962561
R00177__cytop,0.172174,0.172110,0.999626,99.962561


In [149]:
# cleaning data3
data3 = data3.loc[round(data3["flux_c5a"],4) != round(data3["flux_n5a"],4)]

In [180]:
data3.head()

,flux_c5a,flux_n5a,ratio,ratio_100
T_L_Tyrosine_V2__mitmem,0.000000,0.000051,inf,inf
R00848__mito,1.372964,1.638747,1.193583,119.358339
T_UDPG__chlomem,-0.754264,-0.881329,1.168463,116.846293
R01626__chlo,7.377894,7.375132,0.999626,99.962561
R00177__cytop,0.172174,0.172110,0.999626,99.962561


In [151]:
data_125 = data3[data3['ratio_100'] > 125] # overexpressed

# grab those that have less than 75% at ratio_100 (underexpressed)
data_75 = data3[data3['ratio_100'] < 75]


In [154]:
# sort data_125 by ratio_100
data_125 = data_125.sort_values(by=['ratio_100'], ascending=False)

In [155]:
data_75 = data_75.sort_values(by=['ratio_100'], ascending=False)

In [156]:
data_125.to_csv('data/outputs/ratio/2study_c5a_n5a_overexpressed_pfba.csv')
data_75.to_csv('data/outputs/ratio/2study_c5a_n5a_underexpressed_pfba.csv')

In [7]:
dic_reactions2 = getPatwhaysMap(model_c5a)

data_overexpressed = pd.read_csv('data/outputs/ratio/2study_c5a_n5a_overexpressed_pfba.csv')

data_underexpressed = pd.read_csv('data/outputs/ratio/2study_c5a_n5a_underexpressed_pfba.csv')

In [8]:
overexpressed = []
for i in range(len(data_overexpressed)):
    overexpressed.append(data_overexpressed.iloc[i,0])

len(overexpressed)

60

In [9]:
underexpressed = []
for i in range(len(data_underexpressed)):
    underexpressed.append(data_underexpressed.iloc[i,0])
len(underexpressed)

105

In [10]:
path_map = getPatwhaysMap(model_c5a)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in overexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

# print sorted by value
sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)

[('Transporters pathway', 22), ('Biosynthesis of amino acids', 9), ('Glycolysis / Gluconeogenesis', 6), ('Pyruvate metabolism', 5), ('Biosynthesis of cofactors', 5), ('Glyoxylate and dicarboxylate metabolism', 4), ('Methane metabolism', 4), ('2-Oxocarboxylic acid metabolism', 4), ('Cysteine and methionine metabolism', 4), ('Alanine, aspartate and glutamate metabolism', 4), ('Valine, leucine and isoleucine degradation', 3), ('Citrate cycle (TCA cycle)', 3), ('Carbon fixation in photosynthetic organisms', 3), ('Arginine biosynthesis', 3), ('Purine metabolism', 3), ('Glycine, serine and threonine metabolism', 2), ('Carbon fixation pathways in prokaryotes', 2), ('Amino sugar and nucleotide sugar metabolism', 2), ('Galactose metabolism', 2), ('Pentose phosphate pathway', 2), ('Valine, leucine and isoleucine biosynthesis', 2), ('Pantothenate and CoA biosynthesis', 2), ('Glucosinolate biosynthesis', 2), ('Sulfur metabolism', 2), ('Nitrogen metabolism', 2), ('D-Glutamine and D-glutamate metabo

In [17]:
over_fatty = []
for i in range(len(overexpressed)):
    if overexpressed[i] in path_map.keys():

        if 'Biosynthesis of amino acids' in path_map[overexpressed[i]]:
            over_fatty.append(overexpressed[i])
            print(overexpressed[i])


R00734__mito
R01518__cytop
R00355__cytop
R01214__chlo
R01214__cytop
R01827__cytop
R00945__mito
R00586__cytop
R00897__cytop


In [166]:
path_map = getPatwhaysMap(model_c5a)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in underexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

# print sorted by value
sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)

[('Transporters pathway', 27), ('Biosynthesis of amino acids', 12), ('Citrate cycle (TCA cycle)', 11), ('Pyrimidine metabolism', 10), ('Glyoxylate and dicarboxylate metabolism', 7), ('Carbon fixation pathways in prokaryotes', 7), ('Carbon fixation in photosynthetic organisms', 7), ('Pyruvate metabolism', 7), ('Glycine, serine and threonine metabolism', 6), ('2-Oxocarboxylic acid metabolism', 6), ('Purine metabolism', 6), ('Biosynthesis of cofactors', 6), ('Propanoate metabolism', 5), ('Glycolysis / Gluconeogenesis', 4), ('Alanine, aspartate and glutamate metabolism', 4), ('Nitrogen metabolism', 4), ('Carotenoid biosynthesis', 4), ('Arginine biosynthesis', 3), ('One carbon pool by folate', 3), ('Valine, leucine and isoleucine degradation', 2), ('Methane metabolism', 2), ('Arginine and proline metabolism', 2), ('Glutathione metabolism', 2), ('Oxidative phosphorylation', 2), ('Fatty acid metabolism', 2), ('Fatty acid biosynthesis', 2), ('Glycerophospholipid metabolism', 1), ('Pentose and 

In [18]:
und_fatty = []
for i in range(len(underexpressed)):
    if underexpressed[i] in path_map.keys():

        if 'Biosynthesis of amino acids' in path_map[underexpressed[i]]:
            und_fatty.append(underexpressed[i])
            print(underexpressed[i])

R00355__chlo
R01773__cytop
R01771__cytop
R01251__mito
R00200__mito
R01518__mito
R00355__mito
R00267__mito
R01529__chlo
R00734__cytop
R00344__mito
R01466__cytop


In [167]:
# now comparing control vs phosphorus
data4 = pd.DataFrame()
data4['flux_c5a'] = pfba_c5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data4['flux_p5a'] = pfba_p5a.fluxes /  pfba_c5a.fluxes['e_Biomass__cytop']
data4['ratio'] = data4['flux_p5a'] / data4['flux_c5a']
data4 = data4.loc[round(data4["flux_c5a"],4) != round(data4["flux_p5a"],4)]

In [168]:
data4

,flux_c5a,flux_p5a,ratio
T_L_Tyrosine_V2__mitmem,0.000000,0.000052,inf
R00243__mito,-2.618671,-2.489446,0.950652
R00848__mito,1.372964,1.638747,1.193583
R01731__chlo,0.000000,0.130965,inf
T_UDPG__chlomem,-0.754264,0.046132,-0.061162
...,...,...,...
R01845__chlo,43.921718,37.369369,0.850818
R10052__cytop,0.223732,0.229682,1.026594
R02272__chlo,-0.786913,-0.807840,1.026594
e_Carbohydrate__cytop,0.100000,0.102659,1.026594


In [170]:
# abs ratio
data4['ratio'] = abs(data4['ratio'])

#ratio_100
data4['ratio_100'] = data4['ratio'] * 100

In [173]:
# filter dataframe grab those that have more than 125% at ratio_100 (overexpressed)
data_125 = data4[data4['ratio_100'] > 125]

# grab those that have less than 75% at ratio_100 (underexpressed)
data_75 = data4[data4['ratio_100'] < 75]

data_125 = data_125.sort_values(by=['ratio_100'], ascending=False)
data_75 = data_75.sort_values(by=['ratio_100'], ascending=False)

data_125.to_csv('data/outputs/ratio/2study_c5a_p5a_overexpressed_pfba.csv')
data_75.to_csv('data/outputs/ratio/2study_c5a_p5a_underexpressed_pfba.csv')

In [174]:
# read overexpressed reactions from pfba_1_study_overexpressed.csv
data_overexpressed = pd.read_csv('data/outputs/ratio/2study_c5a_p5a_overexpressed_pfba.csv')

# read underexpressed reactions from pfba_1_study_underexpressed.csv
data_underexpressed = pd.read_csv('data/outputs/ratio/2study_c5a_p5a_underexpressed_pfba.csv')

In [175]:
overexpressed = []
for i in range(len(data_overexpressed)):
    overexpressed.append(data_overexpressed.iloc[i,0])
len(overexpressed)

73

In [176]:
underexpressed = []
for i in range(len(data_underexpressed)):
    underexpressed.append(data_underexpressed.iloc[i,0])
len(underexpressed)

145

In [177]:
path_map = getPatwhaysMap(model_c5a)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in overexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

# print sorted by value
sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)

[('Transporters pathway', 19), ('Biosynthesis of amino acids', 11), ('Pyruvate metabolism', 10), ('Glycolysis / Gluconeogenesis', 10), ('Pentose phosphate pathway', 8), ('Purine metabolism', 5), ('Oxidative phosphorylation', 4), ('Fatty acid metabolism', 4), ('Fatty acid biosynthesis', 4), ('Amino sugar and nucleotide sugar metabolism', 3), ('Biosynthesis of cofactors', 3), ('Fructose and mannose metabolism', 3), ('Carbon fixation in photosynthetic organisms', 3), ('2-Oxocarboxylic acid metabolism', 3), ('Glycine, serine and threonine metabolism', 3), ('Pyrimidine metabolism', 3), ('Glyoxylate and dicarboxylate metabolism', 2), ('Galactose metabolism', 2), ('Citrate cycle (TCA cycle)', 2), ('Carbon fixation pathways in prokaryotes', 2), ('Pantothenate and CoA biosynthesis', 2), ('Arginine biosynthesis', 2), ('Nitrogen metabolism', 2), ('Alanine, aspartate and glutamate metabolism', 2), ('Phenylalanine, tyrosine and tryptophan biosynthesis', 2), ('Glycerolipid metabolism', 1), ('Starch 

In [178]:
path_map = getPatwhaysMap(model_c5a)
pathways = []
for pathway in model.groups:
    pathways.append(pathway.name)
res = {}
for key, value in path_map.items():
    if key in underexpressed:
        for reaction_path in value:
            if reaction_path in res.keys():
                res[reaction_path] += 1
            else:
                res[reaction_path] = 1

# print sorted by value
sorted_res = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(sorted_res)

[('Transporters pathway', 50), ('Biosynthesis of amino acids', 17), ('Citrate cycle (TCA cycle)', 15), ('Carbon fixation in photosynthetic organisms', 13), ('Glyoxylate and dicarboxylate metabolism', 12), ('Pyrimidine metabolism', 10), ('2-Oxocarboxylic acid metabolism', 9), ('Glycolysis / Gluconeogenesis', 9), ('Carbon fixation pathways in prokaryotes', 8), ('Pyruvate metabolism', 8), ('Alanine, aspartate and glutamate metabolism', 8), ('Biosynthesis of cofactors', 7), ('Methane metabolism', 5), ('Arginine biosynthesis', 4), ('Nitrogen metabolism', 4), ('Drains pathway', 4), ('Fatty acid metabolism', 4), ('Fatty acid biosynthesis', 4), ('Glycine, serine and threonine metabolism', 3), ('Amino sugar and nucleotide sugar metabolism', 3), ('Pentose phosphate pathway', 3), ('Propanoate metabolism', 3), ('Glutathione metabolism', 3), ('Purine metabolism', 3), ('One carbon pool by folate', 2), ('Galactose metabolism', 2), ('Pentose and glucuronate interconversions', 2), ('Butanoate metabolis